In [ ]:
import pandas as pd

from keys import db2

In [ ]:
%load_ext sql
%config SqlMagic.displaycon = False

conn_string = 'ibm_db_sa://{username}:{password}@{hostname}:{port}/{database}?security=SSL'.format(
    **db2
)

%sql {conn_string}

# Base Course

## Problem 1: Find the total number of crimes recorded in the CRIME table.

In [ ]:
%%sql 

select count(*) as Crimes from chicago_crime_data

## Problem 2: List community areas with per capita income less than 11000.

In [ ]:
%%sql 

select community_area_name, per_capita_income 
from census_data 
where per_capita_income < 11000


## Problem 3: List all case numbers for crimes involving minors?

In [ ]:
%%sql

select case_number, primary_type, description from chicago_crime_data where description like '%MINOR%'

## Problem 4: List all kidnapping crimes involving a child?(children are not considered minors for the purposes of crime analysis)

In [ ]:
%%sql 

select case_number, date, primary_type, description
from chicago_crime_data 
where primary_type = 'KIDNAPPING' and description like '%CHILD%'


## Problem 5: What kind of crimes were recorded at schools?

In [ ]:
%%sql

select distinct primary_type, description from chicago_crime_data where location_description like '%SCHOOL%'

## Problem 6: List the average safety score for all types of schools.

In [ ]:
%%sql
select "Elementary, Middle, or High School", avg(safety_score) as "Avg Safety Score" 
from chicago_public_school 
group by "Elementary, Middle, or High School"

## Problem 7: List 5 community areas with highest % of households below poverty line.

In [ ]:
%%sql 

select community_area_name, percent_households_below_poverty 
from census_data 
order by percent_households_below_poverty	desc 
limit 5

## Problem 8: Which community area(number) is most crime prone?

In [ ]:
%%sql

select community_area_number, count(*) as crime_count 
from chicago_crime_data 
group by community_area_number 
order by crime_count desc 
limit 1


## Problem 9: Use a sub-query to find the name of the community area with highest hardship index.

In [ ]:
%%sql

select community_area_number, community_area_name, hardship_index 
from census_data 
where hardship_index = (select max(hardship_index) from census_data)


## Problem 10: Use a sub-query to determine the Community Area Name with most number of crimes?

In [ ]:
%%sql

select cd.community_area_number, community_area_name, crime_count
from census_data cd, (
  select community_area_number, count(*) as crime_count 
  from chicago_crime_data 
  group by community_area_number 
  order by crime_count desc 
  limit 1
) top_crime
where cd.community_area_number = top_crime.community_area_number


# Honours

## Exercise 1: Using Joins

### Question 1
Write and execute a SQL query to list the school names, community names and average attendance for communities with a hardship index of 98.

In [38]:
%%sql

select name_of_school, ps.community_area_name, average_student_attendance, hardship_index
from chicago_public_school ps
left join census_data census on ps.community_area_number = census.community_area_number
where hardship_index = 98;

Done.


name_of_school,community_area_name,average_student_attendance,hardship_index
George Washington Carver Military Academy High School,RIVERDALE,91.60%,98
George Washington Carver Primary School,RIVERDALE,90.90%,98
Ira F Aldridge Elementary School,RIVERDALE,92.90%,98
William E B Dubois Elementary School,RIVERDALE,93.30%,98


### Question 2

In [39]:
%%sql

select case_number, primary_type, community_area_name
from chicago_crime_data crm
left join census_data cns on crm.community_area_number = cns.community_area_number
where location_description like '%SCHOOL%'

Done.


case_number,primary_type,community_area_name
HK577020,NARCOTICS,Rogers Park
HL725506,BATTERY,Lincoln Square
HH639427,BATTERY,Austin
HS200939,CRIMINAL DAMAGE,Austin
HT315369,ASSAULT,East Garfield Park
HP716225,BATTERY,Douglas
HL353697,BATTERY,South Shore
HS305355,NARCOTICS,Brighton Park
JA460432,BATTERY,Ashburn
HR585012,CRIMINAL TRESPA,Ashburn


## Exercise 2: Creating a View

### Question 1
Write and execute a SQL statement to create a view showing the columns listed in the following table, with new column names as shown in the second column.

Write and execute a SQL statement that returns just the school name and leaders rating from the view.

In [32]:
%%sql

create or replace view NewCrimeView(
  school_id, School_Name, Safety_Rating, Family_Rating, Environment_Rating, Instruction_Rating, Leaders_Rating,Teachers_Rating
) as
select school_id, NAME_OF_SCHOOL, Safety_Icon, Family_Involvement_Icon, Environment_Icon, Instruction_Icon, Leaders_Icon, Teachers_Icon
from chicago_public_school;

select school_id, school_name, leaders_rating from NewCrimeView limit 5
-- select distinct leaders_rating from NewCrimeView

Done.
Done.


school_id,school_name,leaders_rating
610038,Abraham Lincoln Elementary School,Weak
610281,Adam Clayton Powell Paideia Community Academy Elementary School,Weak
610185,Adlai E Stevenson Elementary School,Weak
609993,Agustin Lara Elementary Academy,Weak
610513,Air Force Academy High School,Weak


## Exercise 3: Creating a Stored Procedure

### Question 1
Write the structure of a query to create or replace a stored procedure called UPDATE_LEADERS_SCORE that takes a in_School_ID parameter as an integer and a in_Leader_Score parameter as an integer. Don't forget to use the #SET TERMINATOR statement to use the @ for the CREATE statement terminator.

In [ ]:
%%sql

-- #SET TERMINATOR @
@
create or replace procedure UPDATE_LEADERS_SCORE(in in_school_id int, in in_leader_score int)
begin

end @

### Question 2
Inside your stored procedure, write a SQL statement to update the Leaders_Score field in the CHICAGO_PUBLIC_SCHOOLS table for the school identified by in_School_ID to the value in the in_Leader_Score parameter.

In [ ]:
%%sql

-- #SET TERMINATOR @
@
create or replace procedure UPDATE_LEADERS_SCORE(in in_school_id int, in in_leader_score int)
begin
	update chicago_public_school
	set Leaders_Icon = in_leader_score 
	where school_id = in_school_id;
end @

### Question 3
Inside your stored procedure, write a SQL IF statement to update the Leaders_Icon field in the CHICAGO_PUBLIC_SCHOOLS table for the school identified by in_School_ID using the following information.

In [ ]:
%%sql

-- #SET TERMINATOR @
create or replace procedure UPDATE_LEADERS_SCORE(in in_school_id int, in in_leader_score int)
begin
	declare score varchar(30) default '';
	
	if in_leader_score > 80 then
		set score  = 'Very strong';
	elseif in_leader_score > 60 then
		set score = 'Strong';
	elseif in_leader_score > 40 then
		set score = 'Average';
	elseif in_leader_score > 20 then
		set score = 'Weak';
	else
		set score = 'Very weak';
	end if;
	
	update chicago_public_school
	set Leaders_Icon = score 
	where school_id = in_school_id;
end;
@ --


### Question 4
Run your code to create the stored procedure.

Write a query to call the stored procedure, passing a valid school ID and a leader score of 50, to check that the procedure works as expected.

In [33]:
%%sql

select school_id, school_name, leaders_rating from NewCrimeView where school_id=610038;

call UPDATE_LEADERS_SCORE(610038, 50);

select school_id, school_name, leaders_rating from NewCrimeView where school_id=610038;


Done.
Done.
Done.


school_id,school_name,leaders_rating
610038,Abraham Lincoln Elementary School,Average


## Exercise 4: Using Transactions

### Question 1
Update your stored procedure definition. Add a generic ELSE clause to the IF statement that rolls back the current work if the score did not fit any of the preceding categories.

In [ ]:
%%sql

-- #SET TERMINATOR @
@
create or replace procedure UPDATE_LEADERS_SCORE(in in_school_id int, in in_leader_score int)
MODIFIES SQL DATA
begin
	declare score varchar(30) default '----';
	
	if in_leader_score > 80 then
		set score  = 'Very strong';
	elseif in_leader_score > 60 then
		set score = 'Strong';
	elseif in_leader_score > 40 then
		set score = 'Average';
	elseif in_leader_score > 20 then
		set score = 'Weak';
	elseif in_leader_score > 0 then
		set score = 'Very weak';
	end if;
	
	update chicago_public_school set Leaders_Icon = score  where school_id = in_school_id;
	
	if not in_leader_score between 0 and 100 then
		rollback;
	end if;
end @

### Question 2
Update your stored procedure definition again. Add a statement to commit the current unit of work at the end of the procedure.

In [37]:
%%sql

-- #SET TERMINATOR @
@
create or replace procedure UPDATE_LEADERS_SCORE(in in_school_id int, in in_leader_score int)
MODIFIES SQL DATA
begin
	declare score varchar(30) default '----';
	
	if in_leader_score > 80 then
		set score  = 'Very strong';
	elseif in_leader_score > 60 then
		set score = 'Strong';
	elseif in_leader_score > 40 then
		set score = 'Average';
	elseif in_leader_score > 20 then
		set score = 'Weak';
	elseif in_leader_score > 0 then
		set score = 'Very weak';
	end if;
	
	update chicago_public_school set Leaders_Icon = score where school_id = in_school_id;
	
	if not in_leader_score between 0 and 100 then
		rollback;
	else
		commit;
	end if;
end @

Done.


[]

Run your code to replace the stored procedure.

Write and run one query to check that the updated stored procedure works as expected when you use a valid score of 38.

Write and run another query to check that the updated stored procedure works as expected when you use an invalid score of 101.

In [44]:
%%sql
select school_id, school_name, leaders_rating from NewCrimeView where school_id=610038;

Done.


school_id,school_name,leaders_rating
610038,Abraham Lincoln Elementary School,Average


In [47]:
%%sql
call UPDATE_LEADERS_SCORE(610038, 38);
select school_id, school_name, leaders_rating from NewCrimeView where school_id=610038;

Done.
Done.


school_id,school_name,leaders_rating
610038,Abraham Lincoln Elementary School,Weak


In [48]:
%%sql
call UPDATE_LEADERS_SCORE(610038, 101);
select school_id, school_name, leaders_rating from NewCrimeView where school_id=610038;

Done.
Done.


school_id,school_name,leaders_rating
610038,Abraham Lincoln Elementary School,Weak
